913. Cat and Mouse
Hard
Topics
premium lock icon
Companies
A game on an undirected graph is played by two players, Mouse and Cat, who alternate turns.

The graph is given as follows: graph[a] is a list of all nodes b such that ab is an edge of the graph.

The mouse starts at node 1 and goes first, the cat starts at node 2 and goes second, and there is a hole at node 0.

During each player's turn, they must travel along one edge of the graph that meets where they are.  For example, if the Mouse is at node 1, it must travel to any node in graph[1].

Additionally, it is not allowed for the Cat to travel to the Hole (node 0).

Then, the game can end in three ways:

If ever the Cat occupies the same node as the Mouse, the Cat wins.
If ever the Mouse reaches the Hole, the Mouse wins.
If ever a position is repeated (i.e., the players are in the same position as a previous turn, and it is the same player's turn to move), the game is a draw.
Given a graph, and assuming both players play optimally, return

1 if the mouse wins the game,
2 if the cat wins the game, or
0 if the game is a draw.
 

Example 1:


Input: graph = [[2,5],[3],[0,4,5],[1,4,5],[2,3],[0,2,3]]
Output: 0
Example 2:


Input: graph = [[1,3],[0],[3],[0,2]]
Output: 1
 

Constraints:

3 <= graph.length <= 50
1 <= graph[i].length < graph.length
0 <= graph[i][j] < graph.length
graph[i][j] != i
graph[i] is unique.
The mouse and the cat can always move. 

In [ ]:
class Solution:
    def catMouseGame(self, graph: List[List[int]]) -> int:
        N = len(graph)

        # What nodes could play their turn to
        # arrive at node (m, c, t) ?
        def parents(m, c, t):
            if t == 2:
                for m2 in graph[m]:
                    yield m2, c, 3-t
            else:
                for c2 in graph[c]:
                    if c2:
                        yield m, c2, 3-t

        DRAW, MOUSE, CAT = 0, 1, 2
        color = collections.defaultdict(int)

        # degree[node] : the number of neutral children of this node
        degree = {}
        for m in range(N):
            for c in range(N):
                degree[m,c,1] = len(graph[m])
                degree[m,c,2] = len(graph[c]) - (0 in graph[c])

        # enqueued : all nodes that are colored
        queue = collections.deque([])
        for i in range(N):
            for t in range(1, 3):
                color[0, i, t] = MOUSE
                queue.append((0, i, t, MOUSE))
                if i > 0:
                    color[i, i, t] = CAT
                    queue.append((i, i, t, CAT))

        # percolate
        while queue:
            # for nodes that are colored :
            i, j, t, c = queue.popleft()
            # for every parent of this node i, j, t :
            for i2, j2, t2 in parents(i, j, t):
                # if this parent is not colored :
                if color[i2, j2, t2] is DRAW:
                    # if the parent can make a winning move (ie. mouse to MOUSE), do so
                    if t2 == c: # winning move
                        color[i2, j2, t2] = c
                        queue.append((i2, j2, t2, c))
                    # else, this parent has degree[parent]--, and enqueue if all children
                    # of this parent are colored as losing moves
                    else:
                        degree[i2, j2, t2] -= 1
                        if degree[i2, j2, t2] == 0:
                            color[i2, j2, t2] = 3 - t2
                            queue.append((i2, j2, t2, 3 - t2))

        return color[1, 2, 1]

In [ ]:
class Solution:
    def catMouseGame(self, graph: List[List[int]]) -> int:
        n = len(graph)
        dp = [ [{"c":0, "m":0} for j in range(n)] for i in range(n) ]
        outdegree = [ [{"c":len(graph[j])-(0 in graph[j]), "m":len(graph[i])} for j in range(n)] for i in range(n) ]
        nodes = []
        for j in range(1,n):
            dp[0][j]["m"] = dp[0][j]["c"] = 1
            nodes.extend( [(0,j,"m",1), (0,j,"c",1)] )
            dp[j][j]["m"] =  dp[j][j]["c"] = 2
            nodes.extend( [(j,j,"m",2), (j,j,"c",2)])
        
        while nodes:
            nodes_new = []
            for i, j, move, result in nodes:
                if i==1 and j==2 and move=="m": return result
                premove = "c" if move=="m" else "m"
                if premove=="c":
                    for j_ in graph[j]:
                        if j_==0: continue
                        if dp[i][j_]["c"]: continue
                        if result==2:
                            dp[i][j_]["c"] = 2; nodes_new.append((i,j_,"c",2))
                        else:
                            outdegree[i][j_]["c"]-=1
                            if outdegree[i][j_]["c"]==0:
                                dp[i][j_]["c"] = 1; nodes_new.append((i,j_,"c",1))
                else: # premove=="m"
                    for i_ in graph[i]:
                        if dp[i_][j]["m"]: continue
                        if result==1:
                            dp[i_][j]["m"] = 1; nodes_new.append((i_,j,"m",1))
                        else:
                            outdegree[i_][j]["m"]-=1
                            if outdegree[i_][j]["m"]==0:
                                dp[i_][j]["m"] = 2; nodes_new.append((i_,j,"m",2))
            nodes = nodes_new
        return dp[1][2]["m"]